<h1>Segmenting and Clustering Neighborhoods Part-3</h1>

<h2>1.Description</h2>
For this assignment, you will be required to explore and cluster the neighborhoods in Toronto. 
The data required for this assignment is provided on a  Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, where all the information required for the segmenting and clustering process exist.You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset. From the link http://cocl.us/Geospatial_data, a csv file that contains the geographical coordinates that is latitutudes and longitudes of each neighborhood. Now newly created dataframe contains the geographical coordinates of each neighborhood which helps us to explore the neighborhoods and cluster the neighborhoods.

<h2>2.Scraping Data from Wikipidea Page</h2>

<h4>1.a. Import necessary packages.</h4>

In [57]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # generates random numbers between 0 and 1
import requests# library to handle requests

# map rendering library
!pip install folium
import folium

!pip install lxml #easy handling of XML and HTML files, and can also be used for web scraping.


from bs4 import BeautifulSoup #for getting data out of HTML, XML, and other markup languages.

# import k-means from clustering stage
from sklearn.cluster import KMeans

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

#alternative Python interpreter that provides improvements over the default Python interpreter.
from IPython.display import Image 
from IPython.core.display import HTML
from IPython.display import display_html

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print("Done!!!")

Done!!!


<h5>1.b. Scrape the "raw" table and Tranform the data into a pandas dataframe 'df'</h5>

In [58]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')
table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
   
    if row != [] and row[1] != "Not assigned":
       
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

df=pd.DataFrame(res,columns=["PostalCode", "Borough", "Neighborhood"])
df.head()


,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


<h5>1.c. Cleaning the data by removing "\n" at the end of each string in the each column</h5>

In [59]:
df["Neighborhood"]=df["Neighborhood"].str.replace("\n","")
df["Borough"]=df["Borough"].str.replace("\n","")
df["PostalCode"]=df["PostalCode"].str.replace("\n","")
df.head(11)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


<h5>1.d. Again cleaning data:</h5>
    By ignoring cells with a borough that has 'Not assigned' value. Then next group all neighborhoods with the same postal code.

In [60]:
df=df[df.Borough != 'Not assigned']
df=df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned',df['Borough'],df['Neighborhood'])
df.head(11)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<h2>3. Get the latitude and the longitude coordinates of each neighborhood.</h2>
<h5>3.a. Download the data and read it into a datafframe:</h5>
From the link http://cocl.us/Geospatial_data download a csv file that the geographical coordinates of each postal code and read it into a dataframe 'Geo_df'.

In [62]:
Geo_df = pd.read_csv("https://cocl.us/Geospatial_data")
Geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h5>3.b. Creating a new dataframe :</h5>
Creating a new dataframe df_toronto by merging dataframes df and Geo_df, as per the requirement of the project, that is for easy access of data. 

In [63]:
df_toronto = pd.merge(df, Geo_df, how='left', left_on = 'PostalCode', right_on = 'Postal Code')

df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h5>3.c. Use geopy library to get the latitude and longitude values of Toronto.</h5>
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent toronto_explorer, as shown below.

In [64]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


<h5>3.d.Create a map of Toronto using latitude and longitude values with zoom_start=10</h5>

In [65]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

<h5>3.e. Create a map of Toronto with neighborhoods superimposed on top.</h5>

In [66]:
# add markers to map
for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

<h5>3.f. We are going to work with only the boroughs that contain the word "Toronto"</h5>

In [67]:
df_toronto1 = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto1.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


<h5>3.g.Create a map of Toronto with neighborhoods superimposed on top with zoom_start=12</h5>

In [68]:
map_toronto1 = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_toronto1['Latitude'], 
        df_toronto1['Longitude'], 
        df_toronto1['Borough'], 
        df_toronto1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto1)  

map_toronto1

<h2>4. Define Foursquare Credentials and Version</h2>
On the public repository on Github, I remove this field for the privacy!

In [80]:
CLIENT_ID = '14DRLJLBZCMDQAL4UTGXMX2U4XNK1REXG4VDQNTZVG45DKKX'
CLIENT_SECRET = 'ZBCL2CSRQA2TT1KGNY4QDZHYT5IQUDHXT1Z312ERQLRTFDZO'
VERSION = '20201223'

<h2>5. Explore the first neighborhood in our data frame "df_toronto1"</h2>

In [81]:
neighborhood_name = df_toronto1.loc[0, 'Neighborhood']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

The first neighborhood's name is 'The Beaches'.


<h5>5.a. Get the neighborhood's latitude and longitude values</h5>

In [82]:
neighborhood_latitude = df_toronto1.loc[0, 'Latitude'] 
neighborhood_longitude = df_toronto1.loc[0, 'Longitude'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


<h5>5.b. Get the top 100 venues that are in The Beaches within a radius of 500 meters.</h5>

In [83]:
LIMIT = 100 
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


results = requests.get(url).json()

<h5>5.c. Function that extracts the category of the venue.</h5>

In [84]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<h5>5.d. Now, clean the json and structure it into a pandas dataframe.</h5>

In [85]:

venues =  requests.get(url).json()["response"]['groups'][0]['items']

nearby_venues = json_normalize(venues) 


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


<h2>6. Explore neighborhoods in a part of Toronto City</h2>

<h5>6.a. Create a function again that extracts the category of the venue and clean the json and structure it into a pandas dataframe.</h5>

In [86]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h5>6.b. Now, run the above function on each neighborhood values and create a new dataframe called 'toronto1_venues'</h5>

In [88]:
toronto1_venues = getNearbyVenues(names=df_toronto1['Neighborhood'],
                                   latitudes=df_toronto1['Latitude'],
                                   longitudes=df_toronto1['Longitude']
                                  )

In [89]:
toronto1_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


<h5>6.c. Check for number of venues for each neighborhood</h5>

In [90]:
toronto1_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,59,59,59,59,59,59
Christie,16,16,16,16,16,16
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


<h5>6.d. Check how many unique categories can be curated from all the returned venues</h5>

In [91]:
print('There are {} uniques categories.'.format(len(toronto1_venues['Venue Category'].unique())))

There are 235 uniques categories.


<h2>7. Analyze Each Neighborhood</h2>

In [92]:
# one hot encoding
toronto1_onehot = pd.get_dummies(toronto1_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto1_onehot['Neighborhood'] = toronto1_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto1_onehot.columns[-1]] + list(toronto1_onehot.columns[:-1])
toronto1_onehot = toronto1_onehot[fixed_columns]

toronto1_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h5>7.a. Next, group rows by neighborhood and by taking the mean of the frequency of occurrence of each category</h5>

In [93]:
toronto1_grouped = toronto1_onehot.groupby('Neighborhood').mean().reset_index()
toronto1_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.000000,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,"Business reply mail Processing Centre, South C...",0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,Central Bay Street,0.016949,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.016949,0.0,0.0,0.016949,0.0


<h5>7.b.Check the 10 most common venues in each neighborhood.</h5>

In [94]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto1_grouped['Neighborhood']

for ind in np.arange(toronto1_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto1_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Pharmacy,Beer Bar,Cheese Shop,Seafood Restaurant,Farmers Market,Restaurant,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bakery,Climbing Gym,Burrito Place,Restaurant,Italian Restaurant,Intersection,Bar
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Comic Shop,Burrito Place,Fast Food Restaurant,Farmers Market,Auto Workshop,Restaurant,Smoke Shop,Brewery,Pizza Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Plane,Boutique,Airport,Airport Food Court,Airport Gate,Boat or Ferry
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Salad Place,Thai Restaurant,Bubble Tea Shop,Yoga Studio,Ice Cream Shop


<h2>8. Cluster neighborhoods</h2>
Run k-means to cluster the neighborhood into 5 clusters.

In [95]:
# set number of clusters
kclusters = 5

toronto1_grouped_clustering = toronto1_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto1_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

<h5>8.a. create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood</h5>

In [96]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto1_merged = df_toronto1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto1_merged = toronto1_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto1_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Wine Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Pub,Pizza Place,Lounge,Liquor Store
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Park,Fast Food Restaurant,Pub,Sandwich Place,Board Shop,Brewery,Liquor Store,Italian Restaurant,Restaurant,Burrito Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Bakery,Gastropub,Brewery,Café,American Restaurant,Convenience Store,Seafood Restaurant,Bookstore,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Business Service,Bus Line,Swim School,Park,Comfort Food Restaurant,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop


<h5>8.b. Finally, visualize the resulting clusters</h5>

In [97]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto1_merged['Latitude'], 
        toronto1_merged['Longitude'], 
        toronto1_merged['Neighborhood'], 
        toronto1_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2>8.b. Now, examine each cluster and determine the discriminating venue categories that distinguish each cluster.</h2>

<h4>Cluster-1</h4>

In [98]:
toronto1_merged.loc[toronto1_merged['Cluster Labels'] == 0, toronto1_merged.columns[[1] + list(range(5, toronto1_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Pub,Trail,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Wine Shop


<h4>Cluster-2</h4>

In [99]:
toronto1_merged.loc[toronto1_merged['Cluster Labels'] == 1, toronto1_merged.columns[[1] + list(range(5, toronto1_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Pub,Pizza Place,Lounge,Liquor Store
2,East Toronto,1,Park,Fast Food Restaurant,Pub,Sandwich Place,Board Shop,Brewery,Liquor Store,Italian Restaurant,Restaurant,Burrito Place
3,East Toronto,1,Coffee Shop,Bakery,Gastropub,Brewery,Café,American Restaurant,Convenience Store,Seafood Restaurant,Bookstore,Cheese Shop
5,Central Toronto,1,Sandwich Place,Park,Food & Drink Shop,Pizza Place,Gym / Fitness Center,Hotel,Breakfast Spot,Department Store,Convenience Store,Diner
6,Central Toronto,1,Clothing Store,Coffee Shop,Sporting Goods Shop,Gym / Fitness Center,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Chinese Restaurant,Café
7,Central Toronto,1,Sandwich Place,Dessert Shop,Pizza Place,Gym,Sushi Restaurant,Italian Restaurant,Café,Coffee Shop,Pharmacy,Brewery
9,Central Toronto,1,Coffee Shop,Sushi Restaurant,Pizza Place,Supermarket,Bagel Shop,Bank,Fried Chicken Joint,Restaurant,Pub,American Restaurant
11,Downtown Toronto,1,Coffee Shop,Bakery,Pizza Place,Café,Pub,Restaurant,Italian Restaurant,Sandwich Place,Deli / Bodega,Butcher
12,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Men's Store,Café,Pub,Fast Food Restaurant
13,Downtown Toronto,1,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Theater,Wine Shop,Event Space,Performing Arts Venue


<h4>Cluster-3</h4>

In [100]:
toronto1_merged.loc[toronto1_merged['Cluster Labels'] == 2, toronto1_merged.columns[[1] + list(range(5, toronto1_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,2,Park,Playground,Cupcake Shop,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
10,Downtown Toronto,2,Park,Playground,Trail,Cuban Restaurant,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


<h4>Cluster-4</h4>

In [101]:
toronto1_merged.loc[toronto1_merged['Cluster Labels'] == 3, toronto1_merged.columns[[1] + list(range(5, toronto1_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,3,Garden,Health & Beauty Service,Home Service,Wine Shop,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Dog Run


<h4>Cluster-5</h4>

In [102]:
toronto1_merged.loc[toronto1_merged['Cluster Labels'] == 4, toronto1_merged.columns[[1] + list(range(5, toronto1_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,4,Business Service,Bus Line,Swim School,Park,Comfort Food Restaurant,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop
